# PSEUDO HAMOLTONIAN 

In [1]:
import os
import shutil
import subprocess
import time
import math
import threading
import sys
import torch
import numpy as np
from scipy.ndimage import gaussian_filter
import plotly.graph_objects as go
from concurrent.futures import ThreadPoolExecutor, as_completed
import io

# === Configuration ===
t_min, t_max = 0, 1
render_fps = 240       # High rendering FPS for smooth slow motion
output_fps = 24        # Final video playback FPS
n_qubits = 10          # Number of qubits
system_dim = 2 ** n_qubits         # Hilbert space dimension
s = int(np.sqrt(system_dim))         # Grid size
height_scale = 1
sigma = 1
max_workers = 8
num_frames = int(math.ceil(render_fps * (t_max - t_min)))
chunk_size = 10

# Set up GPU device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# ======================

# --- Quantum Gates Setup ---
I = torch.eye(2, dtype=torch.complex64, device=device)
X = torch.tensor([[0, 1], [1, 0]], dtype=torch.complex64, device=device)
Y = torch.tensor([[0, -1j], [1j, 0]], dtype=torch.complex64, device=device)
Z = torch.tensor([[1, 0], [0, -1]], dtype=torch.complex64, device=device)

def tensor_product(operators):
    """Compute tensor product of multiple operators on GPU"""
    result = operators[0]
    for op in operators[1:]:
        result = torch.kron(result, op)
    return result

# --- Hamiltonian Construction ---
def build_hamiltonian():
    pauli_gates = [I, X, Y, Z]
    num_terms = 10
    H = torch.zeros((system_dim, system_dim), dtype=torch.complex64, device=device)
    
    for _ in range(num_terms):
        gate_indices = torch.randint(0, 4, (n_qubits,))
        paulis = [pauli_gates[i] for i in gate_indices]
        H += torch.randn(1, device=device) * tensor_product(paulis)
    
    return (H + H.mH) / 2  # Hermitian conjugate with mH

# --- Quantum System Initialization ---
print(f"Initializing {system_dim}-dim quantum system on {device}...")
H = build_hamiltonian()

# Create and normalize initial state vector
V_rand = torch.randn(system_dim, dtype=torch.complex64, device=device) + 1j * torch.randn(system_dim, dtype=torch.complex64, device=device)

norm = torch.sqrt(torch.sum(torch.abs(V_rand) ** 2))  # Compute L2 norm
V_rand /= norm  # Normalize

# --- Precompute Evolution ---
print("Precomputing time evolution...")
pre_start = time.time()

# Temporary move H to CPU for eigendecomposition
H_cpu = H.cpu()
eigvals, Q = torch.linalg.eigh(H_cpu)

# Move results back to MPS
Q = Q.to(device)
eigvals = eigvals.to(device)

t_values = torch.linspace(t_min, t_max, num_frames, device=device)
Z_all = torch.empty((s, s, num_frames), device=device)
C_all = torch.empty((s, s, num_frames), device=device)

# --- Modified Phase Calculation Section ---
for i, t in enumerate(t_values):
    sys.stdout.write(f"\rProcessing t = {t:.2f} ({i+1}/{num_frames})")
    sys.stdout.flush()
    
    # Compute time evolution using diagonalization
    phase_factors = torch.exp(-1j * eigvals * t)
    evolved = Q @ (phase_factors * (Q.mH @ V_rand))
    
    # Calculate probabilities on GPU
    prob_data = torch.abs(evolved) ** 2
    prob_matrix = prob_data.view(s, s)
    
    # Process probabilities
    filtered_np = gaussian_filter(prob_matrix.cpu().numpy() * height_scale, sigma=sigma)
    filtered = torch.tensor(filtered_np, device=device)
    
    # Normalization
    fmin, fmax = filtered.min(), filtered.max()
    Z_all[:, :, i] = (filtered - fmin) / (fmax - fmin) if fmax > fmin else 0
    
    # Manual phase calculation using atan2
    evolved_view = evolved.view(s, s)
    real = evolved_view.real
    imag = evolved_view.imag
    phase = (torch.atan2(imag, real) + math.pi) / (2 * math.pi)  # Supported on MPS
    
    # Process phase on CPU (gaussian_filter still needs CPU)
    phase_np = gaussian_filter(phase.cpu().numpy(), sigma=sigma) % 1
    C_all[:, :, i] = torch.tensor(phase_np, device=device)

# Move final results to CPU for visualization
Z_all = Z_all.cpu().numpy()
C_all = C_all.cpu().numpy()

print("\n", end="")
print(f"Precomputed {num_frames} frames in {time.time()-pre_start:.2f}s")

# ... Rest of the visualization code remains the same ...
# --- Visualization Pipeline ---
class ProgressTracker:
    def __init__(self, total_frames):
        self.lock = threading.Lock()
        self.completed = 0
        self.total = total_frames
        self.start_time = time.time()
        
    def update(self, n=1):
        with self.lock:
            self.completed += n
            elapsed = time.time() - self.start_time
            avg_speed = self.completed / elapsed
            progress = self.completed / self.total
            bar = '█' * int(40 * progress) + '─' * (40 - int(40 * progress))
            sys.stdout.write(
                f"\rRendering: |{bar}| {self.completed}/{self.total} "
                f"({progress:.1%}) | {avg_speed:.1f} fps | "
                f"Elapsed: {elapsed:.1f}s"
            )
            sys.stdout.flush()

def create_ffmpeg_pipe(output_filename):
    return subprocess.Popen([
        'ffmpeg', '-y', '-loglevel', 'error',
        '-f', 'image2pipe',
        '-framerate', str(render_fps),
        '-c:v', 'png',
        '-i', '-',
        '-c:v', 'libx264',
        '-crf', '18',
        '-preset', 'slow',
        '-tune', 'animation',
        '-r', str(output_fps),
        '-pix_fmt', 'yuv420p',
        '-movflags', '+faststart',
        output_filename  # Use dynamic filename
    ], stdin=subprocess.PIPE)



# --- Modified Render Function ---
def render_chunk(chunk_range, fig_template, progress):
    buffers = []
    # Create a reusable figure once per chunk
    fig = go.Figure(fig_template)
    
    # Add a surface trace with initial dummy data from the first frame in the chunk
    initial_frame = chunk_range[0]
    surface_trace = go.Surface(
        z=Z_all[:, :, initial_frame],
        surfacecolor=C_all[:, :, initial_frame],
        colorscale='HSV',
        cmin=0,
        cmax=1,
        showscale=False
    )
    fig.add_trace(surface_trace)
    
    # Set the static parts of the layout once
    fig.update_layout(
        scene=dict(
            zaxis=dict(range=[0, 1]),
            aspectratio=dict(x=1, y=1, z=0.7),
            camera_projection=dict(type='orthographic')
        ),
        margin=dict(l=0, r=0, b=0, t=30)
    )
    
    # Update the figure for each frame in the chunk by modifying the trace data and title
    for frame in chunk_range:
        # Update the surface trace data
        fig.data[0].update(
            z=Z_all[:, :, frame],
            surfacecolor=C_all[:, :, frame]
        )
        # Update the title to reflect the current time
        fig.update_layout(
            title_text=f"Time Evolution: t = {t_values[frame]:.2f}",
            title_x=0.5
        )
        
        # Render the updated figure to a PNG image buffer
        buf = io.BytesIO()
        fig.write_image(buf, format='png', scale=4)
        buffers.append(buf.getvalue())
        progress.update(1)
    
    return buffers

    return buffers
if __name__ == "__main__":
    
    shutil.rmtree("frames", ignore_errors=True)
    
    # Visualization template
    fig_template = go.Figure().update_layout(
        scene=dict(
            xaxis_showspikes=False,
            yaxis_showspikes=False,
            zaxis_showspikes=False,
            camera_projection=dict(type='orthographic')
        ),
        paper_bgcolor='white',
        plot_bgcolor='white',
        showlegend=False
    ).to_dict()

        # Generate descriptive filename
    output_filename = (
        f"quantum_q{n_qubits}_"
        f"render{render_fps}_"
        f"output{output_fps}.mp4"
    )
    
    ffmpeg = create_ffmpeg_pipe(output_filename)
    
    progress = ProgressTracker(num_frames)
    
    print(f"\nRendering {num_frames} frames ({s}x{s} grid)")
    render_start = time.time()
    
    try:
        # Create ordered chunks with metadata
        chunks = [(i, range(i, min(i+chunk_size, num_frames))) 
                for i in range(0, num_frames, chunk_size)]
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit tasks with chunk start index
            future_map = {
                executor.submit(
                    render_chunk, 
                    chunk_range, 
                    fig_template, 
                    progress
                ): (start_idx, chunk_range) 
                for start_idx, chunk_range in chunks
            }
            
            # Collect results in completion order
            completed_results = []
            for future in as_completed(future_map):
                start_idx, chunk_range = future_map[future]
                buffers = future.result()
                completed_results.append((start_idx, buffers))
            
            # Sort results by chunk start index
            completed_results.sort(key=lambda x: x[0])
            
            # Write buffers in sorted order
            for _, buffers in completed_results:
                ffmpeg.stdin.write(b''.join(buffers))
                
    finally:
        ffmpeg.stdin.close()
        ffmpeg.wait()
        total_time = time.time() - render_start
        print(f"\n\nTotal render time: {total_time:.1f}s")
        print(f"Average speed: {num_frames/total_time:.1f} FPS")
        print(f"Output: {output_filename}")  # Show actual filename


Initializing 1024-dim quantum system on mps...
Precomputing time evolution...
Processing t = 1.00 (240/240)
Precomputed 240 frames in 2.19s

Rendering 240 frames (32x32 grid)
Rendering: |████████████████████████████████████████| 240/240 (100.0%) | 0.3 fps | Elapsed: 940.5s

Total render time: 943.2s
Average speed: 0.3 FPS
Output: quantum_q10_render240_output24.mp4
